# Neural-Net Training 

In [1]:
# import ipdb
# from tqdm import tqdm
# import os
import numpy as np
# import argparse
# import random
from PIL import Image
# from torch.utils.data import DataLoader
from PIL import Image, ImageDraw

from dataset import Balls_CF_Detection
from dataset_det import COLORS

def show_img(np_array_uint8, out_fn):
    if len(np_array_uint8.shape) == 3:
        img = Image.fromarray(np_array_uint8, 'RGB')
    elif len(np_array_uint8.shape) == 2:
        img = Image.fromarray(np_array_uint8)
    else:
        raise NameError('Unknown data type to show.')

    img.save(out_fn)
    img.show()


def show_bboxes(rgb_array, np_bbox, list_colors, out_fn='./bboxes_on_rgb.png'):
  """ Show the bounding box on a RGB image
  rgb_array: a np.array of shape (H,W,3) - it represents the rgb frame in uint8 type
  np_bbox: np.array of shape (9,4) and a bbox is of type [x1,y1,x2,y2]
  list_colors: list of string of length 9
  """
  assert np_bbox.shape[0] == len(list_colors)
  r=rgb_array.numpy()
  r = np.uint8(r)
  r = np.transpose(r, (1,2,0))

  img_rgb = Image.fromarray(r, 'RGB')
  draw = ImageDraw.Draw(img_rgb)
  N = np_bbox.shape[0]
  for i in range(N):
    color = COLORS[i]
    x_1, y_1, x_2, y_2 = np_bbox[i]
    draw.rectangle(((x_1, y_1), (x_2, y_2)), outline=color, fill=None)

  img_rgb.show()
  img_rgb.save(out_fn)

## Exercice 1

A predicted bounding box is considered correct if it overlaps more than 70% with a ground-truth bounding box, otherwise the bounding box is considered a false positive detection. Multiple detections are penalized. If a system predicts several bounding boxes that overlap with a single ground-truth bounding box, only one prediction is considered correct, the others are considered false positives.


IoU = Area of Overlap / Area of Union

Bounding box: (x,y,width,height) => (x,y) with predefined width & height

https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation

### RESNET

RES NET-18

* transfer learning: pre-trained RESNET
* without transfer learning 

RES NET -> good performance on IMAGE NET
=> compare the performance on our dataset

<b>In practice, very few people train an entire Convolutional Network
from scratch (with random initialization), because it is relatively
rare to have a dataset of sufficient size. Instead, it is common to
pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
contains 1.2 million images with 1000 categories), and then use the
ConvNet either as an initialization or a fixed feature extractor for
the task of interest.</b>

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation

dset = Balls_CF_Detection("data/train/train/")
dval = Balls_CF_Detection("data/train/val/")

image_datasets = { "train": dset, "val": dval }

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = COLORS

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import pickle
pickle_out = open("train_loader","wb")
pickle.dump(dataloaders["train"], pickle_out)
pickle_out.close()

pickle_out = open("test_loader","wb")
pickle.dump(dataloaders["val"], pickle_out)
pickle_out.close()

In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class StandardNet(nn.Module):
    def __init__(self):
        super(StandardNet, self).__init__()
        self.fc1 = nn.Linear(512, 9)

    def forward(self, x):
        x = self.fc1(x)
        output = torch.sigmoid(x)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
        
    batch_idx = 0
    test_loss  = 0
    correct = 0
    total = 0
    for data, target in train_loader:
        data = data.squeeze()
        
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
            
        output = model(data)
        
        loss = nn.BCELoss()(output, target)
        loss.backward()
        optimizer.step()
        
        test_loss += loss
        
        output = (output>0.5).float()
        total += 9 * target.size(0)
        correct += sum(sum(output == target))
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        batch_idx += 1
        
    test_loss /= len(test_loader.dataset) 
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, total,
    100. * correct / total))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in train_loader:
            data = data.squeeze()
            
            data, target = data.to(device), target.to(device)
            output = model(data)            
            test_loss += nn.BCELoss()(output, target)  # sum up batch loss
            
            output = (output>0.5).float()
            total += 9 * target.size(0)
            correct += sum(sum(output == target))
            
        test_loss /= len(test_loader.dataset) 

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, total,
            100. * correct / total))


# Training settings
use_cuda = torch.cuda.is_available()

torch.manual_seed(0)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 100}
test_kwargs = {'batch_size': 100}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = dataloaders["train"]
test_loader = dataloaders["val"]

import pickle
#pickle_out = open("train_loader","wb")
#pickle.dump(train_loader, pickle_out)
#pickle_out.close()

#pickle_out = open("test_loader","wb")
#pickle.dump(test_loader, pickle_out)
#pickle_out.close()

#pickle_in  = open("train_loader","rb")
#train_loader = pickle.load(pickle_in)
#pickle_in.close()

#pickle_in  = open("test_loader","rb")
#test_loader = pickle.load(pickle_in)
#pickle_in.close()

model = StandardNet().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1, rho=0.95, weight_decay=1e-5)

scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()


Train Epoch: 1 [0/10000 (0%)]	Loss: 0.695311
Train Epoch: 1 [100/10000 (1%)]	Loss: 0.790961
Train Epoch: 1 [200/10000 (2%)]	Loss: 0.717612
Train Epoch: 1 [300/10000 (3%)]	Loss: 0.763012
Train Epoch: 1 [400/10000 (4%)]	Loss: 0.744006
Train Epoch: 1 [500/10000 (5%)]	Loss: 0.815862
Train Epoch: 1 [600/10000 (6%)]	Loss: 0.728637
Train Epoch: 1 [700/10000 (7%)]	Loss: 0.802643
Train Epoch: 1 [800/10000 (8%)]	Loss: 0.780090
Train Epoch: 1 [900/10000 (9%)]	Loss: 0.795125
Train Epoch: 1 [1000/10000 (10%)]	Loss: 0.706839
Train Epoch: 1 [1100/10000 (11%)]	Loss: 0.741178
Train Epoch: 1 [1200/10000 (12%)]	Loss: 0.759597
Train Epoch: 1 [1300/10000 (13%)]	Loss: 0.786192
Train Epoch: 1 [1400/10000 (14%)]	Loss: 0.776420
Train Epoch: 1 [1500/10000 (15%)]	Loss: 0.747356
Train Epoch: 1 [1600/10000 (16%)]	Loss: 0.726312
Train Epoch: 1 [1700/10000 (17%)]	Loss: 0.776721
Train Epoch: 1 [1800/10000 (18%)]	Loss: 0.823292
Train Epoch: 1 [1900/10000 (19%)]	Loss: 0.838521
Train Epoch: 1 [2000/10000 (20%)]	Loss: 0.

Train Epoch: 2 [7300/10000 (73%)]	Loss: 0.779384
Train Epoch: 2 [7400/10000 (74%)]	Loss: 0.792848
Train Epoch: 2 [7500/10000 (75%)]	Loss: 0.754564
Train Epoch: 2 [7600/10000 (76%)]	Loss: 0.802815
Train Epoch: 2 [7700/10000 (77%)]	Loss: 0.731155
Train Epoch: 2 [7800/10000 (78%)]	Loss: 0.770375
Train Epoch: 2 [7900/10000 (79%)]	Loss: 0.719367
Train Epoch: 2 [8000/10000 (80%)]	Loss: 0.740763
Train Epoch: 2 [8100/10000 (81%)]	Loss: 0.763133
Train Epoch: 2 [8200/10000 (82%)]	Loss: 0.864588
Train Epoch: 2 [8300/10000 (83%)]	Loss: 0.822343
Train Epoch: 2 [8400/10000 (84%)]	Loss: 0.763012
Train Epoch: 2 [8500/10000 (85%)]	Loss: 0.723858
Train Epoch: 2 [8600/10000 (86%)]	Loss: 0.737912
Train Epoch: 2 [8700/10000 (87%)]	Loss: 0.738291
Train Epoch: 2 [8800/10000 (88%)]	Loss: 0.769343
Train Epoch: 2 [8900/10000 (89%)]	Loss: 0.752817
Train Epoch: 2 [9000/10000 (90%)]	Loss: 0.743764
Train Epoch: 2 [9100/10000 (91%)]	Loss: 0.765909
Train Epoch: 2 [9200/10000 (92%)]	Loss: 0.786397
Train Epoch: 2 [9300

KeyboardInterrupt: 

In [22]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    inp = std * inp + mean
    
    inp = inp/255
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
n = next(iter(dataloaders['train']))
inputs = n[0]
classes = n[1]

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title="Batch Preview")

RuntimeError: output with shape [1, 512, 1] doesn't match the broadcast shape [1, 512, 512, 1]

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                _, preds = torch.topk(outputs,3, 1)
                _, labels_data = torch.topk(labels,3, 1)
                
                # statistics
                running_loss += loss.item() * inputs.size(0)

                for i in range(0,inputs.size(0)): 
                    r=torch.unique(torch.cat((preds[i], labels_data[i]), 0))
                    running_corrects += ((6-len(r))/2)/3.0
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [24]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
    
num_ftrs = model_conv.fc.in_features

# Here the size of each output sample is set to 9.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).

model_conv.fc = nn.Linear(num_ftrs, 9)
model_conv = model_conv.to(device)

criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss()

# Observe that all parameters are being optimized
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [14]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

NameError: name 'criterion' is not defined

In [113]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    error = 0
    total = 0
    #fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)

            outputs = model(inputs).cpu()
            
            outputs = np.array(outputs)
            outputs = np.where(outputs>=0.5, 1, 0)
              
            for j in range(inputs.size()[0]):
                images_so_far += 1
                
                err = abs(np.array(labels - outputs))
                
                error += sum(err[0])
                total += 9
                
                #ax = plt.subplot(num_images//2, 2, images_so_far)
                #ax.axis('off')
                
                #disp_predicted = ""
                
                #for ind in preds[j]:
                #    disp_predicted += COLORS[ind]+ ", "
                
                #ax.set_title('predicted: {}'.format(disp_predicted))
                #imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    print("Accuracy = ")
                    print((total-error)/total)
                    return            
                
        model.train(mode=was_training)

In [114]:
model_conv = torch.load("model_trained")
model_conv.eval()

visualize_model(model_conv, 2000)

Accuracy = 
0.8835555555555555
